## Bic Vu's Capstone Project
#### ML Guild - Apprentice Cohort 11
#### Notebook 2
2023-04-13

In [1]:
import pandas as pd
import pandas_profiling as pf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option("display.precision", 12) # Expand number char shown to 12 instead of scientific notation

/var/folders/2x/fv33h81566l4630885xd8kn40000gp/T/ipykernel_21203/2078206361.py:2: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling as pf


In [2]:
# Create calculate_feature_statistics

from pandas.api.types import is_numeric_dtype

def calculate_feature_statistics(data_frame):
    results = pd.DataFrame()
    for column in data_frame.columns:
        column_data = data_frame[column]
        if not is_numeric_dtype(column_data): continue
        num_zeros = (column_data == 0).sum()
        prop_zeros = num_zeros / len(column_data)
        num_positives = (column_data > 0).sum()
        prop_positives = num_positives / len(column_data)
        num_negatives = (column_data < 0).sum()
        prop_negatives = num_negatives / len(column_data)
        num_missing = column_data.isnull().sum()
        prop_missing = num_missing / len(column_data)
        #most_frequent_value = column_data.mode()[0]
        nunique_value = column_data.nunique()
        prop_nunique = nunique_value / len(column_data)
        min_value = column_data.min()
        max_value = column_data.max()
        median_value = column_data.median()
        feature_stats = pd.Series([num_zeros, prop_zeros, num_positives, prop_positives,
                                   num_negatives, prop_negatives, num_missing, prop_missing,
                                   #most_frequent_value, 
                                   nunique_value, prop_nunique, min_value, max_value, median_value],
                                  index=['num_zeros', 'prop_zeros', 'num_positives', 'prop_positives',
                                         'num_negatives', 'prop_negatives', 'num_missing', 'prop_missing',
                                         #'most_frequent_value', 
                                         'nunique_value', 'prop_nunique', 'min_value', 'max_value', 'median_value'])
        results[column] = feature_stats
    return results.T

In [3]:
# Create shortcut functions to display max rows and columns
def maxcol():
    pd.set_option('display.max_columns', None)
    return

def maxcol_reset():
    pd.reset_option('display.max_columns', None)
    return

def maxrow():
    pd.set_option('display.max_rows', None)
    return

def maxrow_reset():
    pd.reset_option('display.max_rows', None)
    return

### Import raw Social Capital data

In [4]:
filepath = "data/social_capital_zip.csv"
sc_samp = pd.read_csv(filepath, nrows=100)
sc_samp.columns

Index(['zip', 'county', 'num_below_p50', 'pop2018', 'ec_zip', 'ec_se_zip',
       'nbhd_ec_zip', 'ec_grp_mem_zip', 'ec_high_zip', 'ec_high_se_zip',
       'nbhd_ec_high_zip', 'ec_grp_mem_high_zip', 'exposure_grp_mem_zip',
       'exposure_grp_mem_high_zip', 'nbhd_exposure_zip', 'bias_grp_mem_zip',
       'bias_grp_mem_high_zip', 'nbhd_bias_zip', 'nbhd_bias_high_zip',
       'clustering_zip', 'support_ratio_zip', 'volunteering_rate_zip',
       'civic_organizations_zip'],
      dtype='object')

In [5]:
filepath = "data/social_capital_zip.csv"

cols = ["zip", "num_below_p50", "pop2018", "ec_zip", "nbhd_ec_zip", "exposure_grp_mem_zip", "nbhd_exposure_zip", "clustering_zip", 'volunteering_rate_zip', 'civic_organizations_zip']
types = {"zip": str}
sc = pd.read_csv(filepath, usecols=cols, dtype=types)

print(sc.shape)
sc.tail(100)

(23028, 10)


,zip,num_below_p50,pop2018,ec_zip,nbhd_ec_zip,exposure_grp_mem_zip,nbhd_exposure_zip,clustering_zip,volunteering_rate_zip,civic_organizations_zip
22928,99216,2584.053627014160,26107,0.85681999,0.73123002,0.97567999,0.81203002,0.086939998,0.064539999,0.0044000000
22929,99217,2090.080810546875,17616,0.78706002,0.58104002,0.91451001,0.73140001,0.087669998,0.066299997,0.0066300002
22930,99218,963.998716354370,16518,0.87724000,0.78653997,1.01434000,0.90186000,0.088289998,0.064070001,0.0141600000
22931,99223,1982.388222217560,31603,0.96315002,0.93023998,1.08135000,1.07786000,0.085359998,0.068410002,0.0099999998
22932,99224,1187.107990026474,21241,0.86550999,0.69260001,1.01274000,0.93377000,0.088110000,0.069559999,0.0076299999
...,...,...,...,...,...,...,...,...,...,...
23023,99901,1192.299808979034,13818,0.99517000,0.88014001,1.01237000,0.94761997,0.134730000,0.118830000,0.0299900010
23024,99921,365.768661499023,1986,0.87976998,0.74554998,0.88303000,0.81680000,0.155610000,0.084040001,0.0321500000
23025,99925,154.513839721680,927,NaN,NaN,NaN,NaN,0.146579340,0.123960490,0.0277283400
23026,99926,311.014251708984,1635,0.87888002,0.81081003,0.84147000,0.80694002,0.252740000,0.142910000,0.0112500000


In [6]:
sc = sc.dropna().reset_index(drop=True)
print(sc.shape)
sc.head()

(14271, 10)


,zip,num_below_p50,pop2018,ec_zip,nbhd_ec_zip,exposure_grp_mem_zip,nbhd_exposure_zip,clustering_zip,volunteering_rate_zip,civic_organizations_zip
0,1001,995.787467956543,17621,0.88156998,1.51095000,1.12959,1.50590000,0.105720000,0.056499999,0.0108000000
1,1002,1312.117076873779,30066,1.18348000,0.97759998,1.36818,1.20281990,0.103400000,0.149510000,0.0368800010
2,1005,381.519744873047,4991,1.15543000,1.46491000,1.31876,1.46396990,0.105540000,0.158620000,0.0216300000
3,1007,915.396667480469,14967,1.19240000,1.17985000,1.30741,1.23108990,0.103910000,0.130530000,0.0168999990
4,1013,2616.550354003906,23065,0.69744003,0.53930002,0.87842,0.67602003,0.086479999,0.061910000,0.0096899997


In [7]:
sc.dtypes

zip                         object
num_below_p50              float64
pop2018                      int64
ec_zip                     float64
nbhd_ec_zip                float64
exposure_grp_mem_zip       float64
nbhd_exposure_zip          float64
clustering_zip             float64
volunteering_rate_zip      float64
civic_organizations_zip    float64
dtype: object

In [8]:
# Fill in zip codes with less than 5 digits with leading 0
sc["zip"] = sc["zip"].apply(lambda x: x.zfill(5))
sc.head()

,zip,num_below_p50,pop2018,ec_zip,nbhd_ec_zip,exposure_grp_mem_zip,nbhd_exposure_zip,clustering_zip,volunteering_rate_zip,civic_organizations_zip
0,01001,995.787467956543,17621,0.88156998,1.51095000,1.12959,1.50590000,0.105720000,0.056499999,0.0108000000
1,01002,1312.117076873779,30066,1.18348000,0.97759998,1.36818,1.20281990,0.103400000,0.149510000,0.0368800010
2,01005,381.519744873047,4991,1.15543000,1.46491000,1.31876,1.46396990,0.105540000,0.158620000,0.0216300000
3,01007,915.396667480469,14967,1.19240000,1.17985000,1.30741,1.23108990,0.103910000,0.130530000,0.0168999990
4,01013,2616.550354003906,23065,0.69744003,0.53930002,0.87842,0.67602003,0.086479999,0.061910000,0.0096899997


### Opt 1: Zip to Census Tract
Since there are fewer census tracts to zip codes; converting Social Capital zipcode data to census tracts will aggregate some zip codes.

In [9]:
cols = ["zip", "tract"]
zip_ct = pd.read_excel("data/ZIP_TRACT_122021.xlsx", usecols=cols)
zip_ct.head()

,zip,tract
0,683,72023830102
1,683,72125840700
2,683,72125840400
3,683,72125840600
4,683,72121960300


In [10]:
# There are fewer census tracts than zip codes
zip_dup = len(zip_ct["zip"])-len(zip_ct["zip"].drop_duplicates())
print(zip_dup)
tract_dup = len(zip_ct["tract"])-len(zip_ct["tract"].drop_duplicates())
print(tract_dup)

132853
98705


In [11]:
zip_ct = zip_ct.astype(str)
zip_ct.dtypes

zip      object
tract    object
dtype: object

In [12]:
# Format census tracts and zip codes with leading zeroes
zip_ct["tract"] = zip_ct["tract"].apply(lambda x: x.zfill(11))
zip_ct["zip"] = zip_ct["zip"].apply(lambda x: x.zfill(5))
zip_ct.tail()

,zip,tract
172172,99163,53075000500
172173,99163,53075000200
172174,99163,53075001000
172175,78635,48171950100
172176,78635,48031950100


In [13]:
zip_ct = zip_ct.rename(columns={'tract':'census_tract'})

### Convert Social Capital zipcode to census tracts

In [14]:
# Join social capital data with geographic converter on zip
# This will create a lot of duplicates

print(sc.shape)
sc = sc.merge(zip_ct, on="zip", how="left")
print(sc.shape)
sc.head()

(14271, 10)
(117978, 11)


,zip,num_below_p50,pop2018,ec_zip,nbhd_ec_zip,exposure_grp_mem_zip,nbhd_exposure_zip,clustering_zip,volunteering_rate_zip,civic_organizations_zip,census_tract
0,01001,995.787467956543,17621,0.88156998,1.51095,1.12959,1.5059,0.10572,0.056499999,0.0108,25013813209
1,01001,995.787467956543,17621,0.88156998,1.51095,1.12959,1.5059,0.10572,0.056499999,0.0108,25013813208
2,01001,995.787467956543,17621,0.88156998,1.51095,1.12959,1.5059,0.10572,0.056499999,0.0108,25013813204
3,01001,995.787467956543,17621,0.88156998,1.51095,1.12959,1.5059,0.10572,0.056499999,0.0108,25013813207
4,01001,995.787467956543,17621,0.88156998,1.51095,1.12959,1.5059,0.10572,0.056499999,0.0108,25013813206


### Import HMDA Black data and merge with social capital data

In [15]:
# Import HMDA Black data
hmda_black = pd.read_parquet("data/output/hmda_2021_black_group2023-04-14", index=False)
print(hmda_black.shape)
hmda_black.head()

(58363, 8)


,census_tract,approval_black,loan_amount,property_value,income,tract_minority_population_percent,income_level,dti_level
0,01001020100,0.857142857143,253571.428571428580,363571.428571428580,115.142857142857,12.58,upper,20%-<30%
1,01001020200,0.666666666667,135555.555555555562,173888.888888888876,56.666666666667,59.55,middle,42%-<50%
2,01001020300,0.888888888889,125555.555555555562,145000.000000000000,54.944444444444,25.47,middle,42%-<50%
3,01001020400,1.000000000000,234166.666666666657,255000.000000000000,85.333333333333,17.21,middle,36%-<42%
4,01001020500,0.824324324324,213378.378378378373,235945.945945945947,72.918918918919,31.54,upper,42%-<50%


In [20]:
# Join HMDA and Social Capital data on census tract
#### This will generate a lot of duplicate SC data

print(hmda_black.shape)
merged = hmda_black.merge(sc, on="census_tract", how="inner")

print(merged.shape)
maxrow()
merged.tail(40)

(58363, 8)
(94067, 18)


,census_tract,approval_black,loan_amount,property_value,income,tract_minority_population_percent,income_level,dti_level,zip,num_below_p50,pop2018,ec_zip,nbhd_ec_zip,exposure_grp_mem_zip,nbhd_exposure_zip,clustering_zip,volunteering_rate_zip,civic_organizations_zip
94007,56001963100,1.000000000000,2.800000000000e+05,4.350000000000e+05,104.000000000000,19.29,middle,20%-<30%,82072,998.801183700562,16945,1.17139010,1.02408000,1.22750000,1.12798000,0.103480000,0.107330000,0.0441499990
94008,56001963700,1.000000000000,3.250000000000e+05,3.350000000000e+05,101.000000000000,18.38,middle,36%-<42%,82070,1351.962177276611,20336,1.17291000,1.01210000,1.20839000,1.10865000,0.100200000,0.114840000,0.0199700000
94009,56001963700,1.000000000000,3.250000000000e+05,3.350000000000e+05,101.000000000000,18.38,middle,36%-<42%,82072,998.801183700562,16945,1.17139010,1.02408000,1.22750000,1.12798000,0.103480000,0.107330000,0.0441499990
94010,56005000200,1.000000000000,2.800000000000e+05,2.850000000000e+05,99.000000000000,7.66,upper,42%-<50%,82718,1215.131128311157,26340,0.99124002,1.24773000,1.28251000,1.36887000,0.096910000,0.081029996,0.0049000001
94011,56005000400,1.000000000000,1.850000000000e+05,1.850000000000e+05,48.000000000000,22.02,middle,36%-<42%,82716,1553.916566848755,17823,0.90104002,1.08873000,1.20162000,1.21973000,0.096910000,0.086199999,0.0263400000
94012,56005000500,1.000000000000,2.350000000000e+05,2.250000000000e+05,65.000000000000,9.47,upper,42%-<50%,82718,1215.131128311157,26340,0.99124002,1.24773000,1.28251000,1.36887000,0.096910000,0.081029996,0.0049000001
94013,56005000600,1.000000000000,2.150000000000e+05,2.150000000000e+05,54.000000000000,9.77,middle,36%-<42%,82716,1553.916566848755,17823,0.90104002,1.08873000,1.20162000,1.21973000,0.096910000,0.086199999,0.0263400000
94014,56005000700,1.000000000000,1.550000000000e+05,2.050000000000e+05,89.000000000000,8.57,middle,20%-<30%,82716,1553.916566848755,17823,0.90104002,1.08873000,1.20162000,1.21973000,0.096910000,0.086199999,0.0263400000
94015,56005000700,1.000000000000,1.550000000000e+05,2.050000000000e+05,89.000000000000,8.57,middle,20%-<30%,82718,1215.131128311157,26340,0.99124002,1.24773000,1.28251000,1.36887000,0.096910000,0.081029996,0.0049000001
94016,56005000700,1.000000000000,1.550000000000e+05,2.050000000000e+05,89.000000000000,8.57,middle,20%-<30%,82721,381.271057128906,3077,1.06170000,1.10042000,1.12892000,1.09152000,0.109910000,0.072570004,0.0287900010


In [21]:
# Mode creates arrays. Use this fucntion to select the first mode if there are multiple
mode = lambda x: pd.Series.mode(x)[0]

merged2 = merged.groupby('zip', as_index=False).agg({
    "census_tract": mode,
    "approval_black": pd.Series.mean,
    "loan_amount": pd.Series.mean,
    "property_value": pd.Series.mean,
    "income": pd.Series.mean,
    'tract_minority_population_percent': pd.Series.mean,
    'income_level': mode,
    'dti_level': mode,
    "num_below_p50": pd.Series.mean,
    "pop2018": pd.Series.mean,
    "ec_zip": pd.Series.mean,
    "nbhd_ec_zip": pd.Series.mean,
    "exposure_grp_mem_zip": pd.Series.mean,
    "nbhd_exposure_zip": pd.Series.mean,
    "clustering_zip": pd.Series.mean,
    "volunteering_rate_zip": pd.Series.mean,
    "civic_organizations_zip": pd.Series.mean
})
print(merged2.shape)
merged2.tail(20)

(13203, 18)


,zip,census_tract,approval_black,loan_amount,property_value,income,tract_minority_population_percent,income_level,dti_level,num_below_p50,pop2018,ec_zip,nbhd_ec_zip,exposure_grp_mem_zip,nbhd_exposure_zip,clustering_zip,volunteering_rate_zip,civic_organizations_zip
13163,99218,53063010501,0.594444444444,287888.888888888876,425444.444444444438,94.488888888889,13.560000000000,middle,42%-<50%,963.998716354370,16518.0,0.87724000,0.78653997,1.01434000,0.90186000,0.088289998,0.064070001,0.0141600000
13164,99223,53063002900,0.781945327400,256694.320830684446,479349.544394998986,110.258550540369,13.982727272727,upper,42%-<50%,1982.388222217560,31603.0,0.96315002,0.93023998,1.08135000,1.07786000,0.085359998,0.068410002,0.0099999998
13165,99224,53063001000,0.816115702479,236177.685950413230,366067.493112947675,89.778236914601,15.364545454545,middle,36%-<42%,1187.107990026474,21241.0,0.86550999,0.69260001,1.01274000,0.93377000,0.088110000,0.069559999,0.0076299999
13166,99301,53021020100,0.875000000000,222912.698412698403,342378.306878306903,90.223015873016,54.242222222222,middle,20%-<30%,8473.146484375000,78767.0,0.63358998,0.85506999,1.04142000,1.07753000,0.086879998,0.058950000,0.0079699997
13167,99320,53005010701,0.916666666667,434166.666666666686,480000.000000000000,110.916666666667,15.297500000000,upper,42%-<50%,803.009052753448,9644.0,0.83265001,0.75243002,1.00557010,0.88770998,0.092620000,0.079070002,0.0130800000
13168,99324,53071920200,0.750000000000,295000.000000000000,570000.000000000000,155.333333333333,27.230000000000,middle,36%-<42%,931.758079528809,9733.0,0.90433002,0.90862000,1.02184000,0.98303002,0.104800000,0.051380001,0.0248900000
13169,99336,53005010809,0.742222222222,222722.222222222190,400311.111111111124,102.160000000000,35.159000000000,moderate,42%-<50%,4951.424346923828,51998.0,0.76111001,0.57128000,0.88757002,0.72021002,0.081610002,0.068719998,0.0150100000
13170,99337,53005011100,0.683862433862,223564.814814814832,334034.391534391500,73.069444444444,29.128333333333,upper,30%-<36%,2091.320048093796,31088.0,0.81926000,0.79637003,0.98922998,0.88786000,0.086609997,0.065399997,0.0045900000
13171,99344,53001950400,1.000000000000,285000.000000000000,330000.000000000000,146.500000000000,78.385000000000,moderate,36%-<42%,2922.845216751099,17999.0,0.54615998,0.50246000,0.70107001,0.60250998,0.116380000,0.018980000,0.0079199998
13172,99349,53025011402,1.000000000000,255000.000000000000,335000.000000000000,241.000000000000,86.420000000000,moderate,36%-<42%,1108.797973632813,9129.0,0.39276001,0.32447001,0.45952001,0.37579000,0.128470000,0.012650000,0.0094699999


### Export data

In [22]:
# merged.to_parquet(path="data/output/black_features_20230414_allCT", index=None)
# Too many duplicates. Don't use in model

In [23]:
merged2.to_parquet(path="data/output/black_features_20230414_aggZip", index=None)